In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\citiz\.wdm\drivers\chromedriver\win32\87.0.4280.88]


# Step 1 - Scraping

## NASA Mars News

In [3]:
# URL of nasa news page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [4]:
#scrape sight for article
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

articles = soup.find_all('li', class_='slide')
articles
article = articles[0]

In [5]:
#Assign the text to variables that you can reference later
title = article.find('div', class_='content_title').find('a').text
title
para = article.find('div', class_='article_teaser_body').text
para

'During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.'

## JPL Mars Space Images - Featured Image

In [6]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

#scrape sight for article
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

image = soup.find('article', class_='carousel_item')
style = image['style']
arr = style.split("'")
arr[1]
featured_image_url = f'jpl.nasa.gov/' + arr[1]
featured_image_url 

'jpl.nasa.gov//spaceimages/images/wallpaper/PIA19631-1920x1200.jpg'

In [7]:
#browser.quit()

## Mars Facts

In [8]:
#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc
#Use Pandas to convert the data to a HTML table string
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
table = tables[0].to_html()
table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [9]:
from IPython.display import display,HTML

In [10]:
#print html table
display(HTML(table))

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [11]:
#Visit the USGS Astrogeology site
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

base_url = 'https://astrogeology.usgs.gov/'

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
result_items = soup.find('div', class_='results').find_all('div', class_='item')

In [12]:
# obtain high resolution images for each of Mar's hemispheres
#dic list
hemisphere_image_urls = []
#loop to navigate mars sight
for item in result_items:
    #extracting link
    link = item.find('a', class_='itemLink')
    url = base_url + link['href']
    browser.visit(url)
    
    #navigating to image page
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    downloads = soup.find('div', class_='downloads').find_all('a')
    
    #loop through downloads to find images 
    img_url = ''
    for dl in downloads: 
        if '.tif' == dl['href'][-4:]: 
            img_url=dl['href']
    #append to dic list   
    title = soup.find('h2', class_="title").text
    dic = {"title": title, "img_url": img_url}
    hemisphere_image_urls.append(dic)


hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]

In [13]:
browser.quit()